# Trabalho final de Sistemas Baseados em Conhecimento
### Tema: Geração de texto com Markov
### Alunos: Caio Lucas da Silva Chacon (20200025769) e Jonas Gabriel Leite de Araújo (20200007608)

### Dado o início do texto em português, gerar automaticamente algo que faça sentido usando cadeias de Markov

In [1]:
from scipy.sparse import dok_matrix
import numpy as np
import re
from random import random 

# Leitura do arquivo e processamento textual para melhor análise

def read_text(filename):
    """"
    Função que lê um arquivo de texto e retorna o conteúdo em minúsculas
    """

    with open(filename, 'r', encoding="utf8") as file:
        content = file.read().lower()

        content = re.sub(r'[\n,\.;:\"\'_\-\t\—]', ' ', content)
        content = re.sub(r'\s+', ' ', content)

    return content

content = read_text('domCasmurro.txt')
print(content[:100])


dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encont


In [2]:
def get_words(corpus):
    """
    Retorna todas as palavras
    """
    corpus_words = corpus.split(' ')
    corpus_words= [word for word in corpus_words if word != '']
    corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
    return corpus_words # 2185920

def get_distinct_words_count(corpus_words):
    """
    Função que retorna a quantidade de palavras distintas e um dicionário com o índice de cada palavra
    """

    distinct_words = list(set(corpus_words))
    word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
    distinct_words_count = len(list(set(corpus_words)))
 
    return distinct_words_count, word_idx_dict # 32663

### Definição da probabilidade inicial

In [3]:
from collections import Counter

corpus_words = get_words(content)

distinct_words = list(set(corpus_words))
distinct_words_count, word_idx_dict = get_distinct_words_count(corpus_words)

counter_corpus = Counter(corpus_words)

# probabilidade de cada palavra dividido pela quantidade de palavras no corpus
distinct_words_prob = {word: counter_corpus[word]/len(corpus_words) for word in distinct_words}

print("Contagem de palavras distintas: ", distinct_words_count)
print("ID da palavra capitu: ", word_idx_dict['capitu'])
print("Probabilidade da palavra capitu: ", distinct_words_prob['capitu'])

Contagem de palavras distintas:  9129
ID da palavra capitu:  5751
Probabilidade da palavra capitu:  0.004921245119889907


In [4]:
n_grams = 2 # Numero de palavras anteriores para prever a próxima
# Criação de conjuntos de palavras de tamanho n_grams
sets_of_n_words = [ ' '.join(corpus_words[i:i+n_grams]) for i, _ in enumerate(corpus_words[:-n_grams]) ]
len_n_words = len(list(set(sets_of_n_words)))

print(sets_of_n_words[:10])

['dom casmurro', 'casmurro capítulo', 'capítulo primeiro', 'primeiro do', 'do título', 'título uma', 'uma noite', 'noite destas', 'destas vindo', 'vindo da']


### Criação da matriz de transição

In [5]:
# Inicialização de uma matriz esparsa com tamanho len_n_words, distincit_words_count
# para armazenar a contagem de palavras
n_grams_frequency_matrix = dok_matrix((len_n_words, distinct_words_count))

# Criação de um dicionário para mapear os conjuntos de palavras para índices
distinct_sets_of_n_words = list(set(sets_of_n_words))

# Indice pra cada n_gram
n_gram_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_n_words)}

# Preenchimento da matriz com as contagens
for i, word in enumerate(sets_of_n_words[:-n_grams]): # Percorre todos os bigramas
    # Linha da matriz
    n_gram_idx = n_gram_idx_dict[word] # Indice do n_gram
    # Coluna da matriz
    next_word_idx = word_idx_dict[corpus_words[i+n_grams]] # Palavra que aparece logo após o n_grama
    
    # Preenchimento da matriz com a quantidade de vezes que o bigrama ocorre
    n_grams_frequency_matrix[n_gram_idx, next_word_idx] += 1
    
print("(indice_N-grama, indice_proxima_palavra) = qtd_ocorrencias")
print(n_grams_frequency_matrix[:10])

(indice_N-grama, indice_proxima_palavra) = qtd_ocorrencias
  (1, 1373)	1.0
  (4, 7168)	1.0
  (0, 2468)	1.0
  (0, 1268)	1.0
  (9, 7446)	1.0
  (5, 8236)	1.0
  (4, 5842)	1.0
  (8, 7451)	1.0
  (2, 8598)	1.0
  (7, 2023)	1.0
  (6, 6831)	1.0
  (4, 638)	1.0
  (3, 7003)	1.0
  (4, 4212)	1.0


In [6]:
def weighted_choice(weights_array, distinct_words):
    """ 
    Função que retorna um elemento aleatório da sequência de 'objetos',
    a probabilidade dos objetos é ponderada de acordo com a porcentagem.
    """
    sum_of_weights = weights_array.sum() # A soma vai ser sempre igual a 1
    # standardization:
    np.multiply(weights_array, 1000 / sum_of_weights, weights_array)
    weights_array = weights_array.cumsum()
    # print("Pesos acumulados", weights_array)
    x = random()
    # print("Número aleatório", x)

    for i in range(len(weights_array)): # len(weights_array) = len(distinct_words)
        if x < weights_array[i]: # Se o número aleatório for menor que a probabilidade acumulada
            return distinct_words[i]
        
def get_next_word(word_sequence):
    """
    Função que retorna a próxima palavra após a sequência de palavras
    """
    next_word_vector = n_grams_frequency_matrix[n_gram_idx_dict[word_sequence]] # Linha do bigrama
    next_word_probability = next_word_vector/next_word_vector.sum()

    next_word = weighted_choice(next_word_probability.toarray(), distinct_words)
    
    # print("Probabilidade da próxima palavra" , next_word_probability.toarray())
    # print("Próxima palavra" , next_word)    
    return next_word
    
def markov_chain(initial_text, chain_length=30):
    """
    Função que gera um texto de acordo com o texto inicial
    """
    current_words = initial_text.split(' ')
    sentence = initial_text

    for _ in range(chain_length):
        sentence += ' '
        next_word = get_next_word(' '.join(current_words))
        sentence += next_word
        current_words = current_words[1:] + [next_word]
    return sentence

In [7]:
markov_chain('dom casmurro', chain_length=50)

'dom casmurro o receio é que há tais semelhanças inexplicáveis pelo dia adiante e apagou o escrito pádua saiu ao quintal mas as honras dele tio cosme entrou a batê las de um cavaleiro um dandi como então dizíamos montava um belo cavalo alazão firme na cabeça assim me replicou capitu e'

In [8]:
markov_chain('capitu não', chain_length=50)

'capitu não achava tratava se de mim mesmo capítulo verteu ela umas lágrimas que enxugou sem explicar e os olhos dele expliquei nem eu digo vem amanhã sancha não tirava os olhos dele expliquei nem eu digo vem amanhã sancha não tirava os olhos dele expliquei nem eu digo vem amanhã sancha'

In [9]:
markov_chain('agora que', chain_length=50)

'agora que tudo é claro que só há de pagar a dívida contraída outra moeda que valesse tanto ou mais casos estranhos falta me tempo pelo que não digo nada depois da partida sancha quis despedir se dela fique josé dias corrigiu a alegria não tem ambição a separação é indispensável e'

In [10]:
def viterbi(initial_text, n_grams_frequency_matrix, distinct_sets_of_n_words, word_idx_dict, n_grams=1, num_words=100):
    # Quebrar o texto inicial em n-grams
    initial_n_grams = initial_text.split()
    
    # Convertendo a matriz dok para uma matriz densa
    transition_probabilities = n_grams_frequency_matrix.toarray()
    
    # Algoritmo de Viterbi
    sequence = initial_n_grams
    for _ in range(num_words):
        current_state = sequence[-n_grams:]
        current_state_idx = distinct_sets_of_n_words.index(' '.join(current_state))
        #print(current_state_idx)
        
        # Calcular as probabilidades de transição para o próximo estado
        next_state_probs = transition_probabilities[current_state_idx]
        #print(next_state_probs)
        
        # Escolher o próximo estado baseado na probabilidade
        next_state_idx = np.argmax(next_state_probs)
        #print(next_state_idx)
        next_state = distinct_words[next_state_idx].split()[0]
        
        # Adicionar a próxima palavra à sequência
        sequence.append(next_state)
    
    return ' '.join(sequence)

In [18]:
viterbi("capitu refletiu", n_grams_frequency_matrix, distinct_sets_of_n_words, word_idx_dict, n_grams=n_grams, num_words=50)

'capitu refletiu algum tempo e acabou perguntando me se lhe disserem que pode aprender na escola de medicina o ônibus parou e o resto do mundo o que é que não era ainda melhor haver gemido somente sem as saudades confessadas os louvores como um tributo usual cara morta sem agradecimentos o'